In [8]:
import pandas as pd
import numpy as np

# -----------------------------------
# 1. Load Dataset
# -----------------------------------
df = pd.read_csv("supply_chain_data.csv")

In [9]:
df

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,haircare,SKU95,77.903927,65,672,7386.363944,Unknown,15,14,26,...,Mumbai,18,450,26,58.890686,Pending,1.210882,Air,Route A,778.864241
96,cosmetics,SKU96,24.423131,29,324,7698.424766,Non-binary,67,2,32,...,Mumbai,28,648,28,17.803756,Pending,3.872048,Road,Route A,188.742141
97,haircare,SKU97,3.526111,56,62,4370.916580,Male,46,19,4,...,Mumbai,10,535,13,65.765156,Fail,3.376238,Road,Route A,540.132423
98,skincare,SKU98,19.754605,43,913,8525.952560,Female,53,1,27,...,Chennai,28,581,9,5.604691,Pending,2.908122,Rail,Route A,882.198864


In [10]:
# -----------------------------------
# 2. Clean + Standardize Column Names
# -----------------------------------
df.columns = (
    df.columns.str.strip()
               .str.lower()
               .str.replace(" ", "_")
)

In [11]:
# -----------------------------------
# 3. Ensure numeric columns convert properly
# -----------------------------------
numeric_cols = [
    "price",
    "availability",
    "number_of_products_sold",
    "revenue_generated",
    "stock_levels",
    "lead_times",
    "order_quantities",
    "shipping_times",
    "shipping_costs",
    "lead_time",                     
    "production_volumes",
    "manufacturing_lead_time",
    "manufacturing_costs",
    "defect_rates",
    "costs"
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

In [12]:
# -----------------------------------
# 4. Handle missing values
# -----------------------------------
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
df["customer_demographics"] = df["customer_demographics"].fillna("Unknown")
df["supplier_name"] = df["supplier_name"].fillna("Unknown")
df["product_type"] = df["product_type"].fillna("Unknown")

# -----------------------------------
# 5. Save cleaned version
# -----------------------------------
df.to_csv("cleaned_supply_chain.csv", index=False)

print("Step 1 done — dataset cleaned & saved as 'cleaned_supply_chain.csv'.")

Step 1 done — dataset cleaned & saved as 'cleaned_supply_chain.csv'.


In [13]:
import pandas as pd
import numpy as np

# Load cleaned dataset from Step 1
df = pd.read_csv("cleaned_supply_chain.csv")

In [14]:
# -----------------------------------
# 1. Derived Metrics / Feature Engineering
# -----------------------------------

# Total cost = shipping_cost + manufacturing_cost
df["total_unit_cost"] = df["shipping_costs"] + df["manufacturing_costs"]

# Profit per SKU
df["profit"] = df["revenue_generated"] - (df["total_unit_cost"] * df["number_of_products_sold"])

# Profit margin (%)
df["profit_margin"] = (df["profit"] / df["revenue_generated"]) * 100

# Logistics efficiency score
df["logistics_efficiency"] = (
    (1 / (df["shipping_times"] + 1)) * 0.4 +
    (1 / (df["shipping_costs"] + 1)) * 0.3 +
    (1 / (df["lead_times"] + 1)) * 0.3
)

# Supplier performance score
df["supplier_score"] = (
    (1 / (df["manufacturing_lead_time"] + 1)) * 0.5 +
    (1 / (df["defect_rates"] + 1)) * 0.5
)

# Inventory turnover rate
df["inventory_turnover"] = df["number_of_products_sold"] / (df["stock_levels"] + 1)

# Revenue per unit sold
df["revenue_per_unit"] = df["revenue_generated"] / (df["number_of_products_sold"] + 1)


In [15]:
# -----------------------------------
# 2. KPI Summary Table
# -----------------------------------
kpi_summary = pd.DataFrame({
    "Total Revenue": [df["revenue_generated"].sum()],
    "Total Profit": [df["profit"].sum()],
    "Average Profit Margin (%)": [df["profit_margin"].mean()],
    "Average Logistics Efficiency": [df["logistics_efficiency"].mean()],
    "Average Supplier Score": [df["supplier_score"].mean()],
    "Total Units Sold": [df["number_of_products_sold"].sum()],
    "Average Inventory Turnover": [df["inventory_turnover"].mean()],
})

# Save KPI summary
kpi_summary.to_csv("kpi_summary.csv", index=False)

# Save updated dataset
df.to_csv("feature_engineered_supply_chain.csv", index=False)

print("Step 2 done — Feature engineering completed & KPI summary saved.")

Step 2 done — Feature engineering completed & KPI summary saved.


In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load feature-engineered dataset
df = pd.read_csv("feature_engineered_supply_chain.csv")

In [17]:
# Matplotlib style
plt.style.use("ggplot")

# ============================================================
# 1. Revenue by Product Type
# ============================================================
plt.figure(figsize=(10,5))
sns.barplot(data=df, x="product_type", y="revenue_generated")
plt.title("Revenue by Product Type")
plt.xlabel("Product Type")
plt.ylabel("Revenue")
plt.tight_layout()
plt.savefig("revenue_by_product_type.png")
plt.close()

In [18]:
# ============================================================
# 2. Units Sold vs Stock Levels (Scatter Plot)
# ============================================================
plt.figure(figsize=(8,5))
sns.scatterplot(data=df, x="stock_levels", y="number_of_products_sold", hue="product_type", s=80)
plt.title("Units Sold vs Stock Levels")
plt.xlabel("Stock Levels")
plt.ylabel("Units Sold")
plt.tight_layout()
plt.savefig("units_sold_vs_stock_levels.png")
plt.close()


In [19]:
# ============================================================
# 3. Profit Margin Comparison (Bar Chart)
# ============================================================
plt.figure(figsize=(10,5))
sns.barplot(data=df, x="sku", y="profit_margin")
plt.title("Profit Margin by SKU")
plt.xlabel("SKU")
plt.ylabel("Profit Margin (%)")
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig("profit_margin_by_sku.png")
plt.close()

In [20]:
# ============================================================
# 4. Supplier Performance Score
# ============================================================
plt.figure(figsize=(10,5))
sns.barplot(data=df, x="supplier_name", y="supplier_score")
plt.title("Supplier Performance Score")
plt.xlabel("Supplier")
plt.ylabel("Score")
plt.tight_layout()
plt.savefig("supplier_score.png")
plt.close()

In [21]:
# ============================================================
# 5. Correlation Heatmap
# ============================================================
plt.figure(figsize=(12,8))
corr = df.corr(numeric_only=True)
sns.heatmap(corr, cmap="coolwarm", annot=False)
plt.title("Correlation Heatmap")
plt.tight_layout()
plt.savefig("correlation_heatmap.png")
plt.close()

In [25]:
# ============================================================
# 6. Shipping Time vs Shipping Cost
# ============================================================
plt.figure(figsize=(8,5))
sns.scatterplot(
    data=df, 
    x="shipping_times", 
    y="shipping_costs", 
    hue="shipping_carriers",   # FIXED COLUMN NAME
    s=80
)
plt.title("Shipping Time vs Cost")
plt.xlabel("Shipping Times")
plt.ylabel("Shipping Costs")
plt.tight_layout()
plt.savefig("shipping_time_vs_cost.png")
plt.close()


In [23]:
# ============================================================
# 7. Production Efficiency (Production Volume vs Manufacturing Lead Time)
# ============================================================
plt.figure(figsize=(8,5))
sns.scatterplot(data=df, x="production_volumes", y="manufacturing_lead_time", hue="supplier_name")
plt.title("Production Efficiency")
plt.xlabel("Production Volume")
plt.ylabel("Manufacturing Lead Time")
plt.tight_layout()
plt.savefig("production_efficiency.png")
plt.close()


In [27]:
# ============================================================
# 8. Defect Rates Distribution 
# ============================================================
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import numpy as np

plt.figure(figsize=(8,5))

# Histogram
plt.hist(df["defect_rates"], bins=20, alpha=0.6, edgecolor="black", label="Histogram")

# MANUAL KDE — no seaborn, no pandas options
x = df["defect_rates"].dropna()
kde = gaussian_kde(x)
x_range = np.linspace(x.min(), x.max(), 300)
plt.plot(x_range, kde(x_range), linewidth=2, label="KDE Curve")

plt.title("Defect Rates Distribution")
plt.xlabel("Defect Rate")
plt.legend()
plt.tight_layout()
plt.savefig("defect_rate_distribution.png")
plt.close()
